In [6]:
import csv
import urllib
import arcgis
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from arcgis.features.find_locations import find_existing_locations
from arcgis.features import SpatialDataFrame
from arcgis.features import use_proximity
from arcgis.geoenrichment import *
from arcgis.features.manage_data import merge_layers
from arcgis.geoenrichment import standard_geography_query, enrich
import seaborn as sns;
sns.set(color_codes=True)
import math
import datetime
import matplotlib.pyplot as plt
from arcgis.features import summarize_data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
import pandas as pd

In [5]:
ZipsCompiled_df.head()

,Case_Count,OBJECTID,SDEP_SANGIS_ZIPCODE_ZIP,SHAPE,UpdateDate,ZipText,Zip_Code
0,1.0,227,91901,"{""x"": -12990487.759100847, ""y"": 3869543.480973...",2020-04-01 08:00:00,91901,None
1,9.0,228,91902,"{""x"": -13026057.747111531, ""y"": 3851792.785664...",2020-04-01 08:00:00,91902,91902
2,23.0,229,91910,"{""x"": -13031688.913915934, ""y"": 3847142.770844...",2020-04-01 08:00:00,91910,91910
3,21.0,230,91911,"{""x"": -13029970.24674097, ""y"": 3843296.0653008...",2020-04-01 08:00:00,91911,91911
4,20.0,231,91913,"{""x"": -13022988.333377706, ""y"": 3844479.949196...",2020-04-01 08:00:00,91913,91913


In [7]:
ZipsCompiled_df['UpdateDate'] = ZipsCompiled_df['UpdateDate'].astype(str).apply(lambda x: x[:10])
zip_Df = ZipsCompiled_df[['SDEP_SANGIS_ZIPCODE_ZIP','SHAPE','Case_Count','UpdateDate']]
zip_Df['SDEP_SANGIS_ZIPCODE_ZIP'] = zip_Df['SDEP_SANGIS_ZIPCODE_ZIP'].astype(str)
zip_df = pd.DataFrame(columns = zip_Df['UpdateDate'].unique())
zip_df

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-03-31,2020-03-30,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31


In [8]:
zip_Df['Case_Count']=zip_Df['Case_Count'].fillna(0)
for i in list(zip_Df.index):
    value = zip_Df.loc[i]
    zip_df.loc[value[0],value[3]]=int(value[2])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
new_zip_df = zip_df.fillna(0)
new_zip_df['SDEP_SANGIS_ZIPCODE_ZIP'] = new_zip_df.index
shape = zip_Df[['SDEP_SANGIS_ZIPCODE_ZIP','SHAPE']]
shape = shape.drop_duplicates(subset = 'SDEP_SANGIS_ZIPCODE_ZIP')
new_zip_df['SDEP_SANGIS_ZIPCODE_ZIP'] = new_zip_df['SDEP_SANGIS_ZIPCODE_ZIP'].astype(str)
result = new_zip_df.merge(shape, on = 'SDEP_SANGIS_ZIPCODE_ZIP',how = 'inner')
result.head()

,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-03-31,2020-03-30,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,SDEP_SANGIS_ZIPCODE_ZIP,SHAPE
0,1,1,1,1,0,0,1,1,1,1,...,10,10,10,10,10,10,10,11,91901,"{'x': -12990487.759100847, 'y': 3869543.480973..."
1,9,10,10,11,8,8,11,14,16,16,...,38,38,40,40,41,43,43,43,91902,"{'x': -13026057.747111531, 'y': 3851792.785664..."
2,23,28,28,30,21,17,30,34,39,43,...,314,319,327,333,342,353,360,364,91910,"{'x': -13031688.913915934, 'y': 3847142.770844..."
3,21,24,26,29,16,13,29,35,38,47,...,433,443,447,452,456,466,467,472,91911,"{'x': -13029970.24674097, 'y': 3843296.0653008..."
4,20,21,23,25,19,14,25,26,28,32,...,140,142,144,146,149,150,153,153,91913,"{'x': -13022988.333377706, 'y': 3844479.949196..."


In [10]:
#to csv
result ['ZIP'] = result ['SDEP_SANGIS_ZIPCODE_ZIP']
result  = result.drop('SDEP_SANGIS_ZIPCODE_ZIP',axis = 1)
# result.to_csv('result.csv', index = False)

In [11]:
# r = result.spatial.to_featurelayer(title='confirmed cases in zipcode(to 5/31)',gis=gis,tags='dsc198')
# group=gis.groups.search('title:DSC198 Covid-19')
# r.share(groups=group)

In [12]:
result.head()

,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-03-31,2020-03-30,2020-04-05,2020-04-06,2020-04-07,2020-04-08,...,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,SHAPE,ZIP
0,1,1,1,1,0,0,1,1,1,1,...,10,10,10,10,10,10,10,11,"{'x': -12990487.759100847, 'y': 3869543.480973...",91901
1,9,10,10,11,8,8,11,14,16,16,...,38,38,40,40,41,43,43,43,"{'x': -13026057.747111531, 'y': 3851792.785664...",91902
2,23,28,28,30,21,17,30,34,39,43,...,314,319,327,333,342,353,360,364,"{'x': -13031688.913915934, 'y': 3847142.770844...",91910
3,21,24,26,29,16,13,29,35,38,47,...,433,443,447,452,456,466,467,472,"{'x': -13029970.24674097, 'y': 3843296.0653008...",91911
4,20,21,23,25,19,14,25,26,28,32,...,140,142,144,146,149,150,153,153,"{'x': -13022988.333377706, 'y': 3844479.949196...",91913
